In [2]:
from pyts.metrics import dtw as pyts_dtw
from dtw import dtw as dtw_dtw

In [4]:
import numpy as np
import pandas as pd

In [5]:
a = np.array(
    [[1,2,3,4,5,6,7,8,9], [1,2,3,4,5,6,7,8,9]]
)
b = np.array(
    [2,4,6,8,10,12,14,16,18]
)
# pyts_dtw(a, b, dist='absolute')
dtw_dtw(a, b, dist=lambda x, y: np.abs(x - y))

In [15]:
class TimeSeries:
    def __init__(self, data: list):
        self.data = np.array(data)
        
    def __add__(self, other):
        if isinstance(other, TimeSeries):
            return TimeSeries(self.data + other.data)
        elif isinstance(other, (int, float)):
            return TimeSeries(self.data + other)
    
    def __radd__(self, other):
        return self.__add__(other)
    
    def __str__(self):
        return str(self.data)
    
    def __repr__(self):
        return self.__str__()
    
    def append(self, other):
        self.data = np.append(self.data, other)
        return self
    
    def extend(self, other):
        self.data = np.append(self.data, other.data)
        return self
    
    def size(self):
        return len(self.data)

In [21]:
df1 = pd.DataFrame({'name': ['a', 'b', 'c'], 'f1': [TimeSeries([1]), TimeSeries([2]), TimeSeries([3])], 'f2': [TimeSeries([4]), TimeSeries([5]), TimeSeries([6])]})
df2 = pd.DataFrame({'name': ['a', 'b', 'd'], 'f1': [TimeSeries([1]), TimeSeries([2]), TimeSeries([3])], 'f2': [TimeSeries([4]), TimeSeries([5]), TimeSeries([6])]})

In [22]:
merged = pd.merge(df1["name"], df2["name"], on='name', how='outer')
counter = 2
for prefix in ['f1', 'f2']:
    temp = pd.merge(df1[["name", prefix]], df2[["name", prefix]], on='name', how='outer').fillna(TimeSeries([0]))
    merged[prefix] = temp.apply(lambda x: x[prefix + "_x"].extend(x[prefix + "_y"]), axis=1)
    merged[prefix] = merged[prefix].apply(lambda x: TimeSeries([0] * (counter - x.size())).extend(x))

In [23]:
merged

,name,f1,f2
0,a,[1. 1.],[4. 4.]
1,b,[2. 2.],[5. 5.]
2,c,[3. 0.],[6. 0.]
3,d,[0. 3.],[0. 6.]


In [15]:
pd.merge(df1, df2, on='name', how='outer').set_index('name').sum(axis=1).reset_index()

,name,0
0,a,[10]
1,b,[14]
2,c,[9]
3,d,[9]


In [61]:
a = np.arange(20).reshape(5,4)
b = np.arange(5)
c = np.arange(6).reshape(2,3)

In [62]:
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19]])

In [63]:
def init_memval(n_clusters, n_data):
        U = np.random.random((n_clusters, n_data))
        val = sum(U)
        U = np.divide(U, np.dot(np.ones((n_clusters, 1)),
                      np.reshape(val, (1, n_data))))
        return U

In [64]:
data = a

In [138]:
n_clusters = 3
expo = 2
p = 2
alpha = 2
n_data = data.shape[0]
U = init_memval(n_clusters, n_data)

In [66]:
U

array([[0.59375833, 0.34759611, 0.60153116, 0.59608245, 0.34008632],
       [0.04580525, 0.41200158, 0.13229189, 0.02615162, 0.12169136],
       [0.36043642, 0.24040231, 0.26617695, 0.37776593, 0.53822233]])

In [70]:
mf = np.power(U, expo)
center = np.divide(
    np.dot(mf, data), (np.ones((data.shape[1], 1))*sum(mf.T)).T)

In [79]:
center

array([[ 7.26721151,  8.26721151,  9.26721151, 10.26721151],
       [ 5.19504028,  6.19504028,  7.19504028,  8.19504028],
       [10.34142901, 11.34142901, 12.34142901, 13.34142901]])

In [96]:
temp

array([[0.00473374, 0.00926322, 0.00233765],
       [0.02341993, 0.17505516, 0.00621678],
       [0.46556758, 0.03177509, 0.04560138],
       [0.01116107, 0.0053987 , 0.09088074],
       [0.00327819, 0.00214138, 0.00780776]])

In [97]:
dist_part

array([[0.00473374, 0.02341993, 0.46556758, 0.01116107, 0.00327819],
       [0.00926322, 0.17505516, 0.03177509, 0.0053987 , 0.00214138],
       [0.00233765, 0.00621678, 0.04560138, 0.09088074, 0.00780776]])

In [134]:
diff = np.sqrt(np.einsum("ijk->ij", (data[:, None, :] - center) ** 2))
temp = diff ** (1 / (expo - 1))
denominator_ = temp.reshape((data.shape[0], 1, -1)).repeat(
    temp.shape[-1], axis=1
)
denominator_ = temp[:, :, np.newaxis] / denominator_
dist = (1 / denominator_.sum(2)) ** expo

In [135]:
dist

array([[0.10395075, 0.20341612, 0.05133365],
       [0.05538435, 0.41397728, 0.01470169],
       [0.40352699, 0.0275408 , 0.03952464],
       [0.04832406, 0.02337472, 0.39348613],
       [0.08902667, 0.05815402, 0.2120374 ]])

In [126]:
diff

array([[14.53442302, 10.39008057, 20.68285801],
       [ 6.53442302,  2.39008057, 12.68285801],
       [ 1.46557698,  5.60991943,  4.68285801],
       [ 9.46557698, 13.60991943,  3.31714199],
       [17.46557698, 21.60991943, 11.31714199]])

In [140]:
sum(dist * diff) / (alpha * sum(dist))

array([3.1965182 , 3.32589733, 3.61292374])

In [124]:
temp.reshape((data.shape[0], 1, -1)).repeat(
    temp.shape[-1], axis=1
)

array([[[14.53442302, 10.39008057, 20.68285801],
        [14.53442302, 10.39008057, 20.68285801],
        [14.53442302, 10.39008057, 20.68285801]],

       [[ 6.53442302,  2.39008057, 12.68285801],
        [ 6.53442302,  2.39008057, 12.68285801],
        [ 6.53442302,  2.39008057, 12.68285801]],

       [[ 1.46557698,  5.60991943,  4.68285801],
        [ 1.46557698,  5.60991943,  4.68285801],
        [ 1.46557698,  5.60991943,  4.68285801]],

       [[ 9.46557698, 13.60991943,  3.31714199],
        [ 9.46557698, 13.60991943,  3.31714199],
        [ 9.46557698, 13.60991943,  3.31714199]],

       [[17.46557698, 21.60991943, 11.31714199],
        [17.46557698, 21.60991943, 11.31714199],
        [17.46557698, 21.60991943, 11.31714199]]])

In [117]:
denominator_.sum(2)

array([[ 3.53602735,  6.91947871,  1.74618441],
       [ 4.90099544, 36.63310316,  1.30096176],
       [25.86147416,  1.76505116,  2.53307802],
       [ 3.19017394,  1.54311196, 25.97648735],
       [ 2.95074048,  1.92748317,  7.02786418]])

In [95]:
diff = np.zeros((center.shape[0], data.shape[0]))
if center.shape[1] > 1:
    for k in range(center.shape[0]):
        diff[k, :] = np.sqrt(sum(np.power(
            data-np.dot(np.ones((data.shape[0], 1)), np.reshape(center[k, :], (1, center.shape[1]))), 2).T))
else:  # for 1-D data
    for k in range(center.shape[0]):
        diff[k, :] = abs(center[k]-data).T
# dist = diff + 0.0001
dist_part = diff ** (-2 / (expo - 1))


In [104]:
membership = np.equal(U, U.max(axis=0))
dist_part = (np.sqrt(np.einsum("ijk->ij", (data[:, None, :] - center) ** 2)) ** (-2 / (expo - 1))).T
coef_part = np.power(1 - np.multiply(membership, np.divide(1, np.power(n_data, p + 1))), (-2 / (expo - 1)))

In [105]:
numerator_part = np.dot(np.ones((n_clusters, 1)), np.reshape(
                sum(coef_part), (1, coef_part.shape[1]))) * dist_part

In [106]:
np.dot(np.ones((n_clusters, 1)), np.reshape(
                sum(dist_part), (1, dist_part.shape[1]))) * coef_part

array([[0.01659913, 0.20469187, 0.55173652, 0.10918041, 0.01322733],
       [0.01633461, 0.20800666, 0.54294405, 0.10744051, 0.01322733],
       [0.01633461, 0.20469187, 0.54294405, 0.10744051, 0.01344153]])

In [108]:
membership = np.equal(U, U.max(axis=0))
membership_size = np.divide(
    np.sum(np.multiply(membership, U), axis=1), np.power(n_data, p))
relative_size = np.divide(
                np.add(membership_size, 1), n_data).reshape(n_clusters, 1)

In [110]:
index_array = np.argmax(U, axis=0)
interaction_reduction = np.subtract(
    1, np.take_along_axis(relative_size.T, np.expand_dims(index_array, axis=-1), axis=-1)).T

In [111]:
interaction_reduction

array([[0.78566902, 0.79670399, 0.78566902, 0.78566902, 0.79569422]])

In [ ]:
data = np.array([[1, 2], [1, 4], [1, 0], [4, 2], [4, 4], [4, 0]])
n_data = data.shape[0] # Number of data points
n_clusters = 3
p = 2
expo = 2


In [ ]:
# Initialize the partition matrix
U = np.random.random(size=(n_clusters, n_data))
val = sum(U)
U = np.divide(U, np.dot(np.ones((n_clusters, 1)), np.reshape(val, (1, n_data))))


In [ ]:
mf = np.power(U, expo)
np.divide(np.dot(mf, data), (np.ones((data.shape[1], 1))*sum(mf.T)).T)

In [ ]:
np.array([np.sum(data[j, :] * U[i, j] ** 2 for j in range(n_data))
         for i in range(n_clusters)])


In [ ]:
np.sum(1 + U[0, j] / (X.size ** p) for j in range(n))

In [ ]:
S.append(sum([1 + u(i, j) / pow(n, p) for i in range(n_clusters) for j in range(n)]) / X.size)

In [ ]:
v = np.empty((n_clusters, X.shape[1])) # The cluster centers

In [ ]:
v[0] = np.sum([U[0, j] ** 2 * X[j] for j in range(n)])

In [ ]:
[X[j,:] * U[0, j] ** 2 for j in range(n)]

In [ ]:
np.sum(np.array([[1, 2], [1, 4]]), axis=0)

In [ ]:
np.array([np.sum(X[j,:] * U[0, j] ** 2 for j in range(n)) for i in range(3)])